In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import r2_score

from lightgbm import LGBMRegressor

RANDOM_STATE = 42


In [ ]:
df_train_raw = pd.read_csv("data/train.csv")
df_test_raw  = pd.read_csv("data/test.csv")

print("Train:", df_train_raw.shape)
print("Test :", df_test_raw.shape)


In [ ]:
df_train = df_train_raw.drop(columns=["id"])
df_test  = df_test_raw.drop(columns=["id"])


In [ ]:
def remove_duplicate_columns(df):
    mask = df.T.duplicated()
    return df.loc[:, ~mask]

df_train = remove_duplicate_columns(df_train)
df_test  = remove_duplicate_columns(df_test)


In [ ]:
constant_cols = df_train.columns[df_train.nunique() <= 1]

df_train = df_train.drop(columns=constant_cols)
df_test  = df_test.drop(columns=constant_cols, errors="ignore")


In [ ]:
KPI_TARGETS = ["wip", "investissement", "satisfaction"]

X = df_train.drop(columns=KPI_TARGETS)
y = df_train[KPI_TARGETS]

print("X:", X.shape)
print("y:", y.shape)


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=RANDOM_STATE
)


In [ ]:
lgbm = LGBMRegressor(
    n_estimators=1500,
    learning_rate=0.025,
    num_leaves=96,
    min_child_samples=40,
    subsample=0.85,
    colsample_bytree=0.85,
    random_state=RANDOM_STATE,
    n_jobs=-1
)

model_kpis = MultiOutputRegressor(lgbm)
model_kpis.fit(X_train, y_train)


In [ ]:
sat_model = model_kpis.estimators_[2]

importances = pd.Series(
    sat_model.feature_importances_,
    index=X.columns
).sort_values(ascending=False)

TOP_N = 800
top_features = importances.head(TOP_N).index

X_train_top = X_train[top_features]
X_val_top   = X_val[top_features]


In [ ]:
model_kpis.fit(X_train_top, y_train)

y_val_pred = model_kpis.predict(X_val_top)


In [ ]:

y_val_pred_np = np.asarray(y_val_pred)
y_val_np = np.asarray(y_val)

y_val_pred_np = np.clip(y_val_pred_np, 0, None)

sat_bias = np.mean(y_val_np[:, 2] - y_val_pred_np[:, 2])


y_val_pred_np[:, 2] = y_val_pred_np[:, 2] + sat_bias


In [ ]:
r2_per_kpi = r2_score(y_val_np, y_val_pred_np, multioutput="raw_values")

for kpi, r2 in zip(KPI_TARGETS, r2_per_kpi):
    print(f"{kpi:15s} | R2 = {r2:.4f}")

errors = np.abs(y_val_np - y_val_pred_np)
score_per_kpi = (errors < 0.05).mean(axis=0)

for kpi, score in zip(KPI_TARGETS, score_per_kpi):
    print(f"{kpi:15s} | Score ±0.05 = {score:.4f}")

print("\nOverall score:", score_per_kpi.mean())


In [ ]:
X_test_final = df_test.reindex(columns=top_features, fill_value=0)
X_test_final = X_test_final[top_features]


In [ ]:
test_pred_np = np.asarray(model_kpis.predict(X_test_final))

test_pred_np = np.clip(test_pred_np, 0, None)

test_pred_np[:, 2] = test_pred_np[:, 2] + sat_bias


In [ ]:
df_submission = pd.DataFrame(
    test_pred_np,
    columns=["wip", "investissement", "satisfaction"]
)

df_submission.insert(0, "id", df_test_raw["id"].values)

df_submission.head()


In [ ]:
df_submission.to_csv(
    "data/submission_lgbm_simple_sat_bias.csv",
    index=False
)

